# AURA V11 CERBERUS - "NEMESIS" Protocol (Adversarial)

**Objective**: Decouple Toxicity from Negative Sentiment.
**Method**: Adversarial Training with Gradient Reversal Layer (GRL).

### The Logic
- **The Problem**: V10 learned that "Negative Sentiment" = "Toxic".
- **The Fix**: We introduce an **Adversary Head** that tries to guess if a sample comes from the *Toxicity Dataset* (Source 0) or the *Sentiment Dataset* (Source 1).
- **The Trick (GRL)**: When the Adversary tries to learn, we **flip the gradient**. 
    - If BERT extracts features that make it easy to tell Sentiment from Toxicity, it gets **punished**.
    - BERT is forced to learn features that are "Domain Invariant" (true toxicity, not just sadness).

---

In [ ]:
# Cell 1: Imports & Setup
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from torch.optim.lr_scheduler import OneCycleLR
from transformers import BertModel, BertTokenizer
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

In [ ]:
# Cell 2: Configuration V11 (NEMESIS)
CONFIG = {
    'encoder': 'bert-base-uncased',
    'max_length': 128,
    'num_emotion_classes': 7,
    'dropout': 0.5,
    'weight_decay': 0.1,
    'batch_size': 32,
    'gradient_accumulation': 2,
    'epochs': 5,
    'lr_bert': 2e-5,
    'lr_heads': 3e-5,
    'warmup_ratio': 0.1,
    'patience': 5,
    'focal_gamma': 2.0,
    'freezing_epochs': 1,
    'adversarial_lambda': 0.1,      # How much to punish domain knowledge (Start small)
}

EMO_COLS = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']
DATA_DIR = '/kaggle/input/aura-v9-data' 
print("V11 NEMESIS Loaded: Adversarial Protocols Active ⚔️")

In [ ]:
# Cell 3: GRADIENT REVERSAL LAYER (The Magic)
from torch.autograd import Function

class GradientReversal(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.save_for_backward(x, alpha)
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        grad_input = None
        _, alpha = ctx.saved_tensors
        if ctx.needs_input_grad[0]:
            # FLIP THE SIGN! (grad * -alpha)
            grad_input = - alpha * grad_output
        return grad_input, None

def grad_reverse(x, alpha=1.0):
    return GradientReversal.apply(x, torch.tensor(alpha))

print("Gradient Reversal Layer defined.")

In [ ]:
# Cell 4: AURA NEMESIS Model
class AURA_CERBERUS(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.bert = BertModel.from_pretrained(config['encoder'])
        hidden = self.bert.config.hidden_size
        self.dropout = nn.Dropout(config['dropout'])
        
        # Standard Heads
        self.toxicity_head = nn.Linear(hidden, 2)
        self.emotion_head = nn.Linear(hidden, config['num_emotion_classes'])
        self.sentiment_head = nn.Linear(hidden, 2)
        
        # ADVERSARY HEAD (Attempts to predict Domain: Tox vs Sent)
        self.adversary_head = nn.Linear(hidden, 2)
        
        # Uncertainty parameters
        self.log_var_tox = nn.Parameter(torch.tensor(0.0))
        self.log_var_emo = nn.Parameter(torch.tensor(0.0))
        self.log_var_sent = nn.Parameter(torch.tensor(0.0))

        # COURSE INSIGHT: Bias Initialization (Module 3 - Imbalanced Datasets)
        # Initialize Toxicity Head bias to reflect class imbalance (approx 5% toxic)
        # log(pos/neg) = log(0.05/0.95) ≈ -2.94
        # This prevents the initial "shock" and high loss at epoch 0.
        with torch.no_grad():
             self.toxicity_head.bias[0] = 2.94 # Non-Toxic (Majority)
             self.toxicity_head.bias[1] = -2.94 # Toxic (Minority)
        
    def forward(self, input_ids, attention_mask, alpha=1.0):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = self.dropout(outputs.pooler_output)
        
        # 1. Standard Tasks (Normal Gradient)
        tox_out = self.toxicity_head(pooled)
        emo_out = self.emotion_head(pooled)
        sent_out = self.sentiment_head(pooled)
        
        # 2. Adversarial Task (Reversed Gradient)
        # We trick the encoder by reversing the gradient from this head
        adv_pooled = grad_reverse(pooled, alpha)
        domain_out = self.adversary_head(adv_pooled)
        
        return {
            'toxicity': tox_out,
            'emotion': emo_out,
            'sentiment': sent_out,
            'domain': domain_out,
            'log_var_tox': self.log_var_tox,
            'log_var_emo': self.log_var_emo,
            'log_var_sent': self.log_var_sent
        }

print('Model AURA NEMESIS defined (with Bias Init).')

In [ ]:
# Cell 5: Loss Functions
def focal_loss(logits, targets, gamma=2.0, weight=None):
    ce = F.cross_entropy(logits, targets, weight=weight, reduction='none')
    p_t = torch.exp(-ce)
    return ((1 - p_t) ** gamma * ce).mean()

def kendall_loss(task_loss, log_var):
    log_var = torch.clamp(log_var, min=-5.0, max=5.0)
    precision = torch.exp(-log_var)
    return precision * task_loss + log_var

def compute_mtl_loss(outputs, batch, tox_weights, gamma, adv_lambda):
    # A. Primary Tasks Loss (Minimize Error)
    task_loss = torch.tensor(0.0, device=outputs['toxicity'].device)
    
    if batch['task_mask_tox'].sum() > 0:
        l = focal_loss(outputs['toxicity'][batch['task_mask_tox']], batch['tox_label'][batch['task_mask_tox']], gamma, tox_weights)
        task_loss += kendall_loss(l, outputs['log_var_tox'])
        
    if batch['task_mask_emo'].sum() > 0:
        l = F.binary_cross_entropy_with_logits(outputs['emotion'][batch['task_mask_emo']], batch['emo_label'][batch['task_mask_emo']])
        task_loss += kendall_loss(l, outputs['log_var_emo'])
        
    if batch['task_mask_sent'].sum() > 0:
        l = focal_loss(outputs['sentiment'][batch['task_mask_sent']], batch['sent_label'][batch['task_mask_sent']], gamma)
        task_loss += kendall_loss(l, outputs['log_var_sent'])
        
    # B. Adversarial Loss (Minimize Domain Classification Error)
    # Wait, didn't we reverse gradient? YES.
    # In 'forward', we minimize this loss.
    # In 'backward', the gradient flips, so the Encoder effectively Is MAXIMIZING this loss.
    
    # Only train Adversary on Toxicity vs Sentiment samples (Ignore emotion for simplicity or include all)
    # Let's verify we have domain labels.
    adv_loss = F.cross_entropy(outputs['domain'], batch['domain_label'])
    
    # Total = Task + (Lambda * Adversary)
    # Note: Lambda scales the gradient reversal strength implicit in the backward pass.
    # But usually Lambda is applied here. Wait, grad_reverse ALREADY scales by alpha.
    # So we just add the loss.
    return task_loss + adv_loss

print('Adversarial Loss defined.')

In [ ]:
# Cell 6: Datasets (Updated with Domain Label)
class BaseDataset(Dataset):
    def __init__(self, csv_path, tokenizer, max_len):
        self.df = pd.read_csv(csv_path)
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.df)
    def encode(self, text):
        return self.tokenizer.encode_plus(str(text), max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')

class ToxicityDataset(BaseDataset):
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        enc = self.encode(row['text'])
        return {
            'input_ids': enc['input_ids'].flatten(), 'attention_mask': enc['attention_mask'].flatten(),
            'tox_label': torch.tensor(int(row['label']), dtype=torch.long), 'emo_label': torch.zeros(len(EMO_COLS)), 'sent_label': torch.tensor(-1, dtype=torch.long),
            'task': 'toxicity',
            'domain_label': torch.tensor(0, dtype=torch.long) # DOMAIN 0: TOXICITY
        }

class SentimentDataset(BaseDataset):
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        enc = self.encode(row['text'])
        return {
            'input_ids': enc['input_ids'].flatten(), 'attention_mask': enc['attention_mask'].flatten(),
            'tox_label': torch.tensor(-1, dtype=torch.long), 'emo_label': torch.zeros(len(EMO_COLS)), 'sent_label': torch.tensor(int(row['label']), dtype=torch.long),
            'task': 'sentiment',
            'domain_label': torch.tensor(1, dtype=torch.long) # DOMAIN 1: SENTIMENT
        }

class EmotionDataset(BaseDataset):
    def __init__(self, csv_path, tokenizer, max_len, emo_cols):
        super().__init__(csv_path, tokenizer, max_len)
        self.emo_cols = emo_cols
        if 'label_sum' in self.df.columns: self.df = self.df[self.df['label_sum'] > 0].reset_index(drop=True)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        enc = self.encode(row['text'])
        return {
            'input_ids': enc['input_ids'].flatten(), 'attention_mask': enc['attention_mask'].flatten(),
            'tox_label': torch.tensor(-1, dtype=torch.long), 'emo_label': torch.tensor([float(row[c]) for c in self.emo_cols]), 'sent_label': torch.tensor(-1, dtype=torch.long),
            'task': 'emotion',
            'domain_label': torch.tensor(0, dtype=torch.long) # Treat Emotions as 'Toxicity-like' or distinct? Let's assume Domain 0 for simplicity (Source A) or random.
            # SCIENTIFIC DECISION: Emotion data is 'neutral ground'. Let's give it Domain 0 to force it to align with Toxicity context, or 0/1 mix.
            # SAFEST: Ignore Emotion in Adversary (Mask it out) OR treat as Domain 0.
            # Let's set it to 0 so the model learns to align Emotion features with Toxicity features.
        }

def collate_fn(batch):
    tasks = [x['task'] for x in batch]
    return {
        'input_ids': torch.stack([x['input_ids'] for x in batch]),
        'attention_mask': torch.stack([x['attention_mask'] for x in batch]),
        'tox_label': torch.stack([x['tox_label'] for x in batch]),
        'emo_label': torch.stack([x['emo_label'] for x in batch]),
        'sent_label': torch.stack([x['sent_label'] for x in batch]),
        'domain_label': torch.stack([x['domain_label'] for x in batch]),
        'task_mask_tox': torch.tensor([t == 'toxicity' for t in tasks], dtype=torch.bool), 
        'task_mask_emo': torch.tensor([t == 'emotion' for t in tasks], dtype=torch.bool),
        'task_mask_sent': torch.tensor([t == 'sentiment' for t in tasks], dtype=torch.bool)
    }

In [ ]:
# Cell 7: Loaders
tokenizer = BertTokenizer.from_pretrained(CONFIG['encoder'])
tox_train = ToxicityDataset(f'{DATA_DIR}/toxicity_train.csv', tokenizer, CONFIG['max_length'])
emo_train = EmotionDataset(f'{DATA_DIR}/emotions_train.csv', tokenizer, CONFIG['max_length'], EMO_COLS)
sent_train = SentimentDataset(f'{DATA_DIR}/sentiment_train.csv', tokenizer, CONFIG['max_length'])
tox_val = ToxicityDataset(f'{DATA_DIR}/toxicity_val.csv', tokenizer, CONFIG['max_length'])

train_set = ConcatDataset([tox_train, emo_train, sent_train])
train_loader = DataLoader(train_set, batch_size=CONFIG['batch_size'], shuffle=True, collate_fn=collate_fn, num_workers=2)
val_loader = DataLoader(tox_val, batch_size=CONFIG['batch_size'], shuffle=False, collate_fn=collate_fn)
tox_weights = torch.tensor([0.75, 1.5], device=device)
print("Datasets loaded.")

In [ ]:
# Cell 8: Training Engine with FREEZING & ADVERSARIAL Alpha

def train_epoch(model, loader, optimizer, scheduler, config, tox_weights, epoch):
    model.train()
    
    if epoch <= config['freezing_epochs']:
        print(f"❄️ EPOCH {epoch}: ICE AGE. Backbone Frozen.")
        for param in model.bert.parameters(): param.requires_grad = False
    else:
        print(f"🔥 EPOCH {epoch}: FIRE. Backbone Unfrozen.")
        for param in model.bert.parameters(): param.requires_grad = True
            
    total_loss = 0
    optimizer.zero_grad()
    pbar = tqdm(loader, desc='Training')
    
    # DYNAMIC GRL ALPHA: Schedule alpha from 0.0 to 1.0 over epochs to avoid destroying learning early
    # This is standard practice in Domain Adaptation.
    len_loader = len(loader)
    
    for step, batch in enumerate(pbar):
        # Calculate dynamic alpha (p = progress from 0 to 1)
        # p = float(step + (epoch - 1) * len_loader) / (config['epochs'] * len_loader)
        # alpha = 2. / (1. + np.exp(-10 * p)) - 1
        # SIMPLIFIED: Just use fixed lambda for now, pass alpha=1 to layer and scale loss by lambda.
        alpha = 1.0

        for k, v in batch.items(): 
            if isinstance(v, torch.Tensor): batch[k] = v.to(device)
            
        outputs = model(batch['input_ids'], batch['attention_mask'], alpha=alpha)
        loss = compute_mtl_loss(outputs, batch, tox_weights, config['focal_gamma'], config['adversarial_lambda'])
        
        (loss / config['gradient_accumulation']).backward()
        
        if (step + 1) % config['gradient_accumulation'] == 0:
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        
        total_loss += loss.item()
        if step % 50 == 0:
            σ_t = torch.exp(0.5 * model.log_var_tox).item()
            pbar.set_postfix({'loss': loss.item(), 'σ_tox': f'{σ_t:.2f}'})
            
    return total_loss / len(loader)

@torch.no_grad()
def validate(model, loader):
    model.eval()
    preds, labels = [], []
    for batch in tqdm(loader, desc='Validating', leave=False):
        ids, mask = batch['input_ids'].to(device), batch['attention_mask'].to(device)
        # Alpha 0 in validation (no adversarial effect needed for inference)
        out = model(ids, mask, alpha=0.0)
        preds.extend(out['toxicity'].argmax(1).cpu().numpy())
        labels.extend(batch['tox_label'].to(device).cpu().numpy())
    return f1_score(labels, preds, average='macro')

In [ ]:
# Cell 9: Main Loop V11
model = AURA_CERBERUS(CONFIG).to(device)
optimizer = torch.optim.AdamW([
    {'params': model.bert.parameters(), 'lr': CONFIG['lr_bert']},
    {'params': model.toxicity_head.parameters(), 'lr': CONFIG['lr_heads']},
    {'params': model.emotion_head.parameters(), 'lr': CONFIG['lr_heads']},
    {'params': model.sentiment_head.parameters(), 'lr': CONFIG['lr_heads']},
    {'params': model.adversary_head.parameters(), 'lr': CONFIG['lr_heads']}, # NEW
    {'params': [model.log_var_tox, model.log_var_emo, model.log_var_sent], 'lr': CONFIG['lr_heads']}
], weight_decay=CONFIG['weight_decay'])

total_steps = len(train_loader) * CONFIG['epochs'] // CONFIG['gradient_accumulation']
scheduler = OneCycleLR(optimizer, max_lr=[CONFIG['lr_bert']] + [CONFIG['lr_heads']]*5, total_steps=total_steps, pct_start=CONFIG['warmup_ratio'])

best_val_f1 = 0.0
for epoch in range(1, CONFIG['epochs'] + 1):
    avg_loss = train_epoch(model, train_loader, optimizer, scheduler, CONFIG, tox_weights, epoch)
    val_f1 = validate(model, val_loader)
    print(f'Epoch {epoch}: Loss={avg_loss:.4f}, Val F1={val_f1:.4f}')
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), 'aura_v11_nemesis.pt')
        print('>>> NEW BEST MODEL <<<')